V tomto notebooku sa venujeme vytvaraniu rozhodovacich stromov za pomoci kniznice scikit-learn. Najprv si data upravujeme pomocou OneHotEncoderu, pricom nahradzujeme kategoricke data za numericke. Potom vytvarame rozhodovaci strom bez specifikovania hyperparametrov, skumame jeho vysledky. Nasledne sa venujeme optimalizacii hyperparametrov pomocou GridSearch-u a skumame najlepsie nastavenie hyperparametrov.

In [52]:
import scipy.stats as stats
import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute._iterative import IterativeImputer
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import dateutil.parser as parser
from sklearn.impute import SimpleImputer as Imputer, SimpleImputer
from sklearn.neighbors import NearestNeighbors as neigh
from sklearn.neighbors import KNeighborsRegressor as neigh
from sklearn.model_selection import train_test_split as split
from sklearn import metrics
from math import sqrt
import json
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from missingpy import KNNImputer, MissForest
from sklearn.tree import DecisionTreeClassifier
import category_encoders as ce
import copy
from sklearn.tree import export_graphviz
from graphviz import Source
from IPython.display import SVG
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid

Odstranenie nepotrebnych stlpcov a nacitanie.
Ziskanie kategorickych dat a pretransformovanie ich pomocou OneHotEncoder-u.
Odstranenie povodnych stlpcov a pridanie novo vytvorenych.

In [53]:
merged_data = pd.read_csv('trained_datas.csv',index_col=0)
# merged_data.info()
merged_data = merged_data.drop(['name', 'address', 'date_of_birth'], axis = 1)
categoric = merged_data.select_dtypes(include = ['object'])

encoder = ce.OneHotEncoder()
encoder.fit(categoric)
transformed = encoder.transform(categoric)

merged_data = merged_data.drop(categoric, axis = 1)
merged_data = merged_data.merge(transformed, left_index=True, right_index=True)
merged_data

,education-num,capital-gain,fnlwgt,hours-per-week,capital-loss,mean_glucose,std_glucose,kurtosis_glucose,skewness_glucose,mean_oxygen,...,native-country_37,native-country_38,native-country_39,workclass_1,workclass_2,workclass_3,workclass_4,workclass_5,workclass_6,workclass_7
0,1.595991,-0.307122,1.389199,2.451870,-0.227166,95.351562,41.828816,1.280503,3.351277,14.103679,...,0,0,0,1,0,0,0,0,0,0
1,-0.060364,-0.307122,0.215814,1.190627,-0.227166,111.164062,51.802174,0.267679,-0.141940,2.902174,...,0,0,0,0,1,0,0,0,0,0
2,-0.060364,-0.307122,0.276509,0.774574,-0.227166,14.460938,29.387737,5.600391,34.223119,57.382107,...,0,0,0,0,1,0,0,0,0,0
3,-0.060364,-0.307122,-0.437771,-2.727383,-0.227166,107.796875,41.562709,0.510415,1.672603,1.910535,...,0,0,0,0,1,0,0,0,0,0
4,-0.060364,-0.307122,0.055621,2.451870,-0.227166,111.484375,53.068350,0.459786,-0.116594,1.599498,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3928,-0.447286,-0.307122,0.674189,-0.049761,-0.227166,103.578125,45.166638,0.552355,0.777646,21.821070,...,0,0,0,0,1,0,0,0,0,0
3929,1.166759,-0.307122,-0.156044,1.190627,-0.227166,27.468750,37.283110,4.430580,20.844460,7.814381,...,0,0,0,0,1,0,0,0,0,0
3930,-0.060364,-0.307122,1.665610,-0.049761,-0.227166,100.859375,32.731324,0.251239,3.491732,4.968227,...,0,0,0,0,1,0,0,0,0,0
3931,-2.176550,-0.307122,0.274627,0.361026,-0.227166,40.398438,33.748458,3.933903,19.294670,96.349498,...,0,0,0,0,1,0,0,0,0,0


Vytvorenie rozhodovacieho stromu a natrenovanie ho. 

In [54]:
cls = DecisionTreeClassifier(criterion = 'entropy')

X = merged_data[merged_data.columns.difference(['class'])]
y = merged_data['class']
cls = cls.fit(X, y)


Vizualizacia rozhodovacieho stromu.

In [55]:
graph = Source(export_graphviz(cls, 
                               out_file=None,
                               feature_names=X.columns,
                               class_names=['no diabetes', 'yes diabetes'],
                               filled = True))

# display(SVG(graph.pipe(format='svg')))

# from IPython.display import HTML # toto je tu len pre to aby sa mi obrazok zmestil na obrazovku
# style = "<style>svg{width:70% !important;height:70% !important;}</style>"
# HTML(style)

Upravenie validacnych dat priprava na pouztie v strome, pouzitie encoderu a transformovanie.

In [56]:
val_merged_data = pd.read_csv('validate_datas.csv',index_col=0)
# val_merged_data.info()
val_merged_data = val_merged_data.drop(['name', 'address', 'date_of_birth'], axis = 1)
categoric = val_merged_data.select_dtypes(include = ['object'])
transformed = encoder.transform(categoric)

val_merged_data = val_merged_data.drop(categoric, axis = 1)
val_merged_data = val_merged_data.merge(transformed, left_index=True, right_index=True)

Zistili sme, ze validacne data obsahuju v stlpci 'class' dve hodnoty NaN. Zaznamy obsahujuce tieto
hodnoty sme sa rozhodli odtranit z datasetu.

In [57]:
val_merged_data = val_merged_data.dropna()
val_merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 1310
Columns: 117 entries, education-num to workclass_7
dtypes: float64(15), int64(102)
memory usage: 1.2 MB


Prezenieme vsetky validacne data stromov a skontrolujeme aku presnost sme dosiahli. 
Ulozime si predikciu a nasledne ju budeme kontrolovat s povodnymi hodnotami stlpca 'class'.

In [58]:
prediction = cls.predict(val_merged_data[val_merged_data.columns.difference(['class'])])

In [59]:
print(classification_report(val_merged_data['class'],prediction, digits=3))

              precision    recall  f1-score   support

         0.0      0.969     0.943     0.956       963
         1.0      0.852     0.916     0.883       346

    accuracy                          0.936      1309
   macro avg      0.911     0.930     0.919      1309
weighted avg      0.938     0.936     0.937      1309



### 4. bod - Optimalizacia hyperparametrov

Vyuzijeme GridSearch, pricom budeme vyuzivat 4 rozne hyperparametre, ktore mozeme menit a to su: 
* max depth - zvolili sme interval <2, 27> (ide o maximalnu hlbku stromu)
* criterion - zvolili sme bud gini alebo entropy (ide o funkciu na meranie kvality rozdelovania intervalov)
* min sample leaf - zvolili sme interval <1, 5> (ide o minimalny pocet hodnot, ktore musi obsahovat kazdy list)
* splitter - zvolili sme best alebo random (ide o strategiu rozdelovania pri kazdom uzle)
* max_leaf_nodes - zvolili sme interval <20, 25> (ide o maximalny pocet listov, ktory moze mat jeden uzol)


GridSearch sme si zvolili, aby sme odhalili najlepsie riesenie. Kedze teraz uz mame silny stroj (i7 8750H, 6 jadier, base speed 2.20GHz, max speed 4.1GHz) a nie su kladene podmienky na casovu narocnost. Vdaka tomu, ze si to mozeme dovolit, sme nevyuzili RandomSearch.

In [60]:
clf = DecisionTreeClassifier

cv_params = {'max_depth': list(range(2,20,1)), 'criterion': ['gini', 'entropy'], 'min_samples_leaf': list(range(1,6,1)), 'splitter':  ['best', 'random'], 'max_leaf_nodes' : list(range(20,26,1))}
ind_params = {'random_state': 0}
optimization = GridSearchCV(clf(**ind_params), 
                            cv_params, 
                             scoring = 'f1_macro', cv = 10, n_jobs = -1, verbose=True)

Rozdelili sme si trenovaci dataset a ten sme priradili do X a y. Pricom v X su data bez stplca 'class' a y obsahuje stlpec 'class' teda hodnoty, ktore chceme predikovat.

In [61]:
X = merged_data[merged_data.columns.difference(['class'])]
y = merged_data['class']

Natrenujeme si stromy, pricom skusame rozne parametre, ktore sme si zadefinovali v GridSearch. A taktiez sledujume cas tohto trenovania.

In [62]:
%%time
optimization.fit(X, y)

Fitting 10 folds for each of 2160 candidates, totalling 21600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 1240 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 3240 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 6040 tasks      | elapsed:   23.2s
[Parallel(n_jobs=-1)]: Done 9640 tasks      | elapsed:   40.7s
[Parallel(n_jobs=-1)]: Done 14040 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 19240 tasks      | elapsed:  1.6min


Wall time: 1min 50s


[Parallel(n_jobs=-1)]: Done 21600 out of 21600 | elapsed:  1.8min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=0,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
  

Vdaka cv results mozeme vidiet jednotlive vysledky, ktore dosahuje GridSearch pri jednotlivych kombinaciach parametrov.

In [63]:
# optimization.cv_results_

Zistime si parametre, ktore dosahuju najlepsie vysledky. Z tych, co sme zadali do GridSearch to su:
* max depth = 7
* criterion = entropy
* min samples leaf = 2
* splitter = random
* max leaf nodes = 25


Tato kombinacia parametrov dosiahla skore 94.56% na trenovacom datasete.

In [64]:
optimization.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=7,
                       max_features=None, max_leaf_nodes=25,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='random')

In [65]:
optimization.best_score_

0.9456087715868917

Najprv si rozdelime validacne data, rovnakym sposobom ako trenovacie.

In [66]:
X = val_merged_data[merged_data.columns.difference(['class'])]
y = val_merged_data['class']

Spustime predikciu nad validacnymi datami.

In [67]:
prediction = optimization.best_estimator_.predict(X)

Hyperparametre, ktore boli optimalizovane pomocou GridSearch, naslednym vytvorenim stromu s tymito hyperparametrami sme dosiahli precision 95.5%, recall 94.4% a accuracy 96.1%.

Skusali sme aj ine parametre 
* 'max_features': [4] (ide o pocet ficur, ktore sa zvazuju pri hladani najlepsieho rozdelenia)

ktore vsak nijako neovplyvnili vysledok v pozitivny prospech, praveze ho zhorsili a dosiahli sme s nimi precision 90.9% a recall 89.4%.

Po vyskusani odstranenia jednotlivych definovanych parametrov (min samples leaf, splitter) sa precision a recall nijako nezmenili, no pokial sme odstranili max leaf, precision nam klesla na 93.9% a recall velmi mierne stupol na 93.9%.



In [68]:
print(classification_report(y ,prediction, digits=3))

              precision    recall  f1-score   support

         0.0      0.967     0.980     0.974       963
         1.0      0.943     0.908     0.925       346

    accuracy                          0.961      1309
   macro avg      0.955     0.944     0.949      1309
weighted avg      0.961     0.961     0.961      1309

